In [20]:
import pandas as pd 

In [21]:
ds = pd.read_csv("antwerp_ds_modified.csv", index_col=0) 

In [22]:
ds

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 64,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,-200,2017-12-28 16:14:07.247000+00:00,12,1.02,51.216736,4.412130
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-11-28 12:01:02.380000+00:00,7,0.57,51.232037,4.405185
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-14 09:05:22.832000+00:00,9,1.00,51.202763,4.390477
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,-200,-200,-200,-200,-200,2018-01-17 09:11:49.222000+00:00,12,0.62,51.194584,4.417887
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-10 14:47:01.379000+00:00,12,1.00,51.207203,4.403470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,-200,-103,-106,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-04 10:57:15.856000+00:00,12,0.61,51.201839,4.414310
123524,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-03 23:17:34.799000+00:00,8,1.10,51.233677,4.416892
123525,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-15 10:13:56.383000+00:00,12,0.82,51.215370,4.412519
123526,-200,-97,-200,-200,-200,-200,-200,-200,-200,-92,...,-200,-200,-200,-200,-200,2018-01-12 23:33:09.229000+00:00,12,0.61,51.194252,4.418354


In [23]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [41]:
def get_weather_info(lat, long, time:str):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": long,
        "start_hour": time,
        "end_hour": time,
        "hourly": ["temperature_2m", 
                "relative_humidity_2m",
                "dew_point_2m", 
                "apparent_temperature", 
                "precipitation", 
                "rain", 
                "snowfall", 
                "snow_depth", 
                "weather_code", 
                "pressure_msl", 
                "surface_pressure", 
                "cloud_cover", 
                "cloud_cover_low", 
                "cloud_cover_mid", 
                "cloud_cover_high", 
                "et0_fao_evapotranspiration", 
                "vapour_pressure_deficit", 
                "wind_speed_10m", 
                "wind_speed_100m", 
                "wind_direction_10m", 
                "wind_direction_100m", 
                "wind_gusts_10m", 
                "shortwave_radiation", 
                "direct_radiation",
                "diffuse_radiation", 
                "direct_normal_irradiance", 
                "global_tilted_irradiance", 
                "terrestrial_radiation", 
                "shortwave_radiation_instant",
                "direct_radiation_instant", 
                "diffuse_radiation_instant", 
                "direct_normal_irradiance_instant", 
                "global_tilted_irradiance_instant", 
                "terrestrial_radiation_instant"]
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(22).ValuesAsNumpy()
    hourly_direct_radiation = hourly.Variables(23).ValuesAsNumpy()
    hourly_diffuse_radiation = hourly.Variables(24).ValuesAsNumpy()
    hourly_direct_normal_irradiance = hourly.Variables(25).ValuesAsNumpy()
    hourly_global_tilted_irradiance = hourly.Variables(26).ValuesAsNumpy()
    hourly_terrestrial_radiation = hourly.Variables(27).ValuesAsNumpy()
    hourly_shortwave_radiation_instant = hourly.Variables(28).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(29).ValuesAsNumpy()
    hourly_diffuse_radiation_instant = hourly.Variables(30).ValuesAsNumpy()
    hourly_direct_normal_irradiance_instant = hourly.Variables(31).ValuesAsNumpy()
    hourly_global_tilted_irradiance_instant = hourly.Variables(32).ValuesAsNumpy()
    hourly_terrestrial_radiation_instant = hourly.Variables(33).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
    hourly_data["direct_radiation"] = hourly_direct_radiation
    hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
    hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
    hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
    hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
    hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
    hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
    hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
    hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
    hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe

In [42]:
get_weather_info(ds["Latitude"][0], ds["Longitude"][0], "2017-12-28T16:14")

Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 20.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,2017-12-28 16:14:00+00:00,3.148,74.749611,-0.902,-1.583164,0.0,0.0,0.0,0.0,1.0,...,5.0,51.51376,7.999999,21.081083,0.0,0.0,0.0,0.0,0.0,0.0
